In [1]:
# 导入椭圆曲线算法
from ecdsa import SigningKey, SECP256k1, VerifyingKey, BadSignatureError
import binascii
import base64
from hashlib import sha256


class Wallet:
    """
        钱包
    """
    def __init__(self):
        """
            钱包初始化时基于椭圆曲线生成一个唯一的秘钥对，代表区块链上一个唯一的账户
        """
        self._private_key = SigningKey.generate(curve=SECP256k1)
        self._public_key = self._private_key.get_verifying_key()

    @property
    def address(self):
        """
            这里通过公钥生成地址
        """
        h = sha256(self._public_key.to_pem())
        return base64.b64encode(h.digest())

    @property
    def pubkey(self):
        """
            返回公钥字符串
        """
        return self._public_key.to_pem()

    def sign(self, message):
        """
            生成数字签名
        """
        h = sha256(message.encode('utf8'))
        return binascii.hexlify(self._private_key.sign(h.digest()))

    
def verify_sign(pubkey, message, signature):
    """
        验证签名
    """
    verifier = VerifyingKey.from_pem(pubkey)
    h = sha256(message.encode('utf8'))
    return verifier.verify(binascii.unhexlify(signature), h.digest())

In [2]:
# 新建一个钱包

w = Wallet()

In [3]:
# 打印钱包地址

w.address

b'dKrpZUamm1B2avx22I/OOEkUg+wsnrsRu7qVYTn5wuI='

In [4]:
# 打印钱包公钥

w.pubkey

b'-----BEGIN PUBLIC KEY-----\nMFYwEAYHKoZIzj0CAQYFK4EEAAoDQgAE9bZoTMnsq/14DkQA/yEQG4EsvwYUhlRl\nYoQ+yj1To97nOsMuwPkBa8zQVGpBCF/oJs0X4zCqBk41bCWeOYgFug==\n-----END PUBLIC KEY-----\n'

In [5]:
# 测试数据

data = "交易数据"

In [6]:
# 生成签名

sig = w.sign(data)

# 打印签名
print(sig)

b'00a40310e47fd032662f787b8e201d8edbb3ec031aabea3f287b494f9b2b21c52ac1144e187ac54eea4264e54153ef6778cac7a15e7660bcb5d0750c27d18a50'


In [7]:
# 验证签名

verify_sign(w.pubkey, data, sig)

True

In [8]:
import json

class Transaction:
    """
    交易的结构
    """
    def __init__(self, sender, recipient, amount):
        """
            初始化交易，设置交易的发送方、接收方和交易数量
        """
        if isinstance(sender, bytes):
            sender = sender.decode('utf-8')
        self.sender = sender            # 发送方
        if isinstance(recipient, bytes):
            recipient = recipient.decode('utf-8')
        self.recipient = recipient      # 接收方
        self.amount = amount            # 交易数量
        
    def set_sign(self, signature, pubkey):
        """
            为了便于验证这个交易的可靠性，需要发送方输入他的公钥和签名
        """
        self.signature = signature      # 签名
        self.pubkey = pubkey            # 发送方公钥
        
    def __repr__(self):
        """
            交易大致可分为两种，一是挖矿所得，而是转账交易
            挖矿所得无发送方，以此进行区分显示不同内容
        """
        if self.sender:
            s = "从 %s 转至 %s %d个加密货币" % (self.sender, self.recipient, self.amount)
        else:
            s = "%s 挖矿获取%d个加密货币" % (self.recipient, self.amount)
        return s


class TransactionEncoder(json.JSONEncoder):
    """
    定义Json的编码类，用来序列化Transaction
    """
    def default(self, obj):
        if isinstance(obj, Transaction):
            return obj.__dict__
        else:
            return json.JSONEncoder.default(self, obj)

In [9]:
import hashlib
from datetime import datetime


class Block:
    """
        区块结构
            prev_hash:      父区块哈希值
            transactions:           交易对
            timestamp:      区块创建时间
            hash:           区块哈希值
            Nonce:        随机数
    """
    def __init__(self, transactions, prev_hash):
        # 将传入的父哈希值和数据保存到类变量中
        self.prev_hash = prev_hash    
        self.transactions = transactions
        # 获取当前时间
        self.timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        # 设置Nonce和哈希的初始值为None
        self.nonce = None
        self.hash = None
        
    def __repr__(self):
        return "区块内容：%s\n哈希值: %s" % (json.dumps(self.transactions), self.hash)

In [10]:

class ProofOfWork:
    """
        工作量证明
    """
    def __init__(self, block, miner, difficult=5):
        self.block = block
        
        # 定义工作量难度，默认为5，表示有效的哈希值以5个“0”开头
        self.difficulty = difficult

        self.miner = miner
        # 添加挖矿奖励
        self.reward_amount = 1

    def mine(self):
        """
            挖矿函数
        """
        i = 0
        prefix = '0' * self.difficulty
        
        
        # 添加奖励
        t = Transaction(
                sender="",
                recipient=self.miner.address,
                amount=self.reward_amount,
            )
        sig = self.miner.sign(json.dumps(t, cls=TransactionEncoder))
        t.set_sign(sig, self.miner.pubkey)
        self.block.transactions.append(t)

        while True:
            message = hashlib.sha256()
            message.update(str(self.block.prev_hash).encode('utf-8'))
            # 更新区块中的交易数据
            # message.update(str(self.block.data).encode('utf-8'))
            message.update(str(self.block.transactions).encode('utf-8'))
            message.update(str(self.block.timestamp).encode('utf-8'))
            message.update(str(i).encode("utf-8"))
            digest = message.hexdigest()
            if digest.startswith(prefix):
                self.block.nonce = i
                self.block.hash = digest
                return self.block
            i += 1

    def validate(self):
        """
            验证有效性
        """
        message = hashlib.sha256()
        message.update(str(self.block.prev_hash).encode('utf-8'))
        # 更新区块中的交易数据
        # message.update(str(self.block.data).encode('utf-8'))
        message.update(json.dumps(self.block.transactions).encode('utf-8'))
        message.update(str(self.block.timestamp).encode('utf-8'))
        message.update(str(self.block.nonce).encode('utf-8'))
        digest = message.hexdigest()

        prefix = '0' * self.difficulty
        return digest.startswith(prefix)

In [11]:
class BlockChain:
    """
        区块链结构体
            blocks:        包含的区块列表
    """
    def __init__(self):
        self.blocks = []

    def add_block(self, block):
        """
        添加区块
        """
        self.blocks.append(block)
    
    def print_list(self):
        print("区块链包含区块个数: %d\n" % len(self.blocks))
        for block in self.blocks:
            print("上个区块哈希：%s" % block.prev_hash)
            print("区块内容：%s" % block.transactions)
            print("区块哈希：%s" % block.hash)
            print("\n") 

In [12]:
def get_balance(user):
    balance = 0
    for block in blockchain.blocks:
        for t in block.transactions:
            if t.sender == user.address.decode():
                balance -= t.amount
            elif t.recipient == user.address.decode():
                balance += t.amount
    return balance

In [13]:
# 初始化区块链
blockchain = BlockChain()

# 创建三个钱包，一个属于alice，一个属于tom，剩下一个属于bob
alice = Wallet()
tom = Wallet()
bob = Wallet()

# 打印当前钱包情况
print("alice: %d个加密货币" % (get_balance(alice)))
print("tom: %d个加密货币" % (get_balance(tom)))
print("bob: %d个加密货币" % (get_balance(bob)))

alice: 0个加密货币
tom: 0个加密货币
bob: 0个加密货币


In [14]:
# alice生成创世区块，并添加到区块链中

new_block1 = Block(transactions=[], prev_hash="")
w1 = ProofOfWork(new_block1, alice)
genesis_block = w1.mine()
blockchain.add_block(genesis_block)

In [15]:
# 显示alice当前余额

print("alice: %d个加密货币" % (get_balance(alice)))

alice: 1个加密货币


In [16]:
# alice 转账给 tom 0.3个加密货币
transactions = []
new_transaction = Transaction(
    sender=alice.address,
    recipient=tom.address,
    amount=0.3
)
sig = alice.sign(str(new_transaction))
new_transaction.set_sign(sig, alice.pubkey)

In [17]:
# bob 在网络上接收到这笔交易，进行验证没问题后生成一个新的区块并添加到区块链中

if verify_sign(new_transaction.pubkey, 
                  str(new_transaction),
                   new_transaction.signature):
    
    # 验证交易签名没问题，生成一个新的区块
    print("验证交易成功")
    new_block2 = Block(transactions=[new_transaction], prev_hash="")
    print("生成新的区块...")
    w2 = ProofOfWork(new_block2, bob)
    block = w2.mine()
    print("将新区块添加到区块链中")
    blockchain.add_block(block)
else:
    print("交易验证失败！")

验证交易成功
生成新的区块...
将新区块添加到区块链中


In [20]:
# 打印当前钱包情况
print("alice: %.1f个加密货币" % (get_balance(alice)))
print("tom: %.1f个加密货币" % (get_balance(tom)))
print("bob: %d个加密货币" % (get_balance(bob)))

alice: 0.7个加密货币
tom: 0.3个加密货币
bob: 1个加密货币
